# Saving and Loading Models

TensorFlow에서 모형을 저장하고 불러오는 방법을 알아본다.

## Import Resources

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


In [17]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [18]:
print('Using:')
print('TensorFlow version:', tf.__version__)
print('tf.keras version:', tf.keras.__version__)
print('Running on GPU' if tf.test.is_gpu_available() else 'GPU device not found. Running on CPU')

Using:
TensorFlow version: 2.3.0
tf.keras version: 2.4.0
GPU device not found. Running on CPU


## Load the Dataset

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [20]:
# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

In [21]:
# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

## Explore the Dataset

In [22]:
num_training_examples = len(x_train)
num_validation_examples = len(x_val)
num_test_examples = len(x_test)

print('There are {:,} images in the training set'.format(num_training_examples))
print('There are {:,} images in the validation set'.format(num_validation_examples))
print('There are {:,} images in the test set'.format(num_test_examples))

There are 50,000 images in the training set
There are 10,000 images in the validation set
There are 10,000 images in the test set


In [23]:
x_train = x_train.reshape(num_training_examples, 28, 28,1)
x_val = x_val.reshape(num_validation_examples, 28, 28,1)
x_test = x_test.reshape(num_test_examples, 28, 28,1)

training_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_set = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [24]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [25]:
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()

plt.imshow(image, cmap=plt.cm.binary)
plt.title(class_names[int(label)])
plt.colorbar()
plt.show()

## Create Pipeline

In [26]:
def normalize(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 64

training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)
validation_batches = validation_set.cache().batch(batch_size).map(normalize).prefetch(1)
testing_batches = test_set.cache().batch(batch_size).map(normalize).prefetch(1)

## Build and Train the Model



In [27]:
layer_neurons = [512, 256, 128]

dropout_rate = 0.5

model = tf.keras.Sequential()

model.add(tf.keras.layers.Flatten(input_shape=(28,28,1)))

for neurons in layer_neurons:
    model.add(tf.keras.layers.Dense(neurons, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))

model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

In [28]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

EPOCHS = 4

history = model.fit(training_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/4
782/782 [==============================] - 5s 6ms/step - loss: 1.0831 - accuracy: 0.5739 - val_loss: 0.6314 - val_accuracy: 0.7619
Epoch 2/4
782/782 [==============================] - 4s 5ms/step - loss: 0.6619 - accuracy: 0.7569 - val_loss: 0.5248 - val_accuracy: 0.7975
Epoch 3/4
782/782 [==============================] - 4s 5ms/step - loss: 0.5787 - accuracy: 0.7890 - val_loss: 0.4807 - val_accuracy: 0.8160
Epoch 4/4
782/782 [==============================] - 4s 5ms/step - loss: 0.5250 - accuracy: 0.8137 - val_loss: 0.4487 - val_accuracy: 0.8326


## Saving and Loading Models

TensorFlow 에서는 모형을 2가지 형태로 저장할 수 있다. 하나는 TensorFlow의 SavedModel format 이고 다른 하나는 HDF5 files format으로 Keras models type 이다.

### Saving and Loading Models in HDF5 Format

 `.save(filepath)` method를 사용하여 모형을 저장한다. 예를 들면 다음과 같다.

```python
my_model.save('./test_model.h5')
```

파일의 확장자는 반드시 `.h5` 으로 한다. 

 HDF5 file 은 다음과 같은 내용을 저장한다.:

* model의 구조.
* model의 weight values
* model의 훈련과 관련된 설정 `compile` method의 parameter값.
* optimizer에 대한 정보


기본적으로 `.save()` method는 에러 없이 기존의 파일을 덮어쓰기 한다.`overwrite=False` 을 설정하면 파일 덮어쓰기 전에 prompt한다.

In [29]:
t = time.time()

saved_keras_model_filepath = './{}.h5'.format(int(t))

model.save(saved_keras_model_filepath)

일단 저장된 모형은 `tf.keras.models.load_model(filepath)` 을 통해 불러들인다. 

In [30]:
reloaded_keras_model = tf.keras.models.load_model(saved_keras_model_filepath)

reloaded_keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

`reloaded_keras_model`은 원래 `model` 과 같은 복사본이 된다. 다음과 같은 명령어로 Prediction 을 수행할 수 있다.

In [31]:
for image_batch, label_batch in testing_batches.take(1):
    prediction_1 = model.predict(image_batch)
    prediction_2 = reloaded_keras_model.predict(image_batch)
    difference = np.abs(prediction_1 - prediction_2)
    print(difference.max())

0.0


결과가 0.0 인것은 같은 이미지에 대해 같은 prediction을 수행한 결과이다.

### Saving and Loading TensorFlow SavedModels

TensorFlow 의 **SavedModel** format로 모형을 저장하기 위해서 `tf.saved_model.save(model, export_dir)` function을 사용한다. 예를 들면 다음과 같다.

```python
tf.saved_model.save(my_model, './saved_models')
```

주의할 것은 파일의 이름을 쓰는 것이 아니라 모형이 저장될 directory명을 쓴다는 것이다. 
왜냐면 SavedModels 은 단일한 file로 저장되는 것이 아니기 때문이다. `tf.saved_model.save()` function은 `assets` folder, `variables` folder 그리고 `saved_model.pb` 을 생성한다.

SavedModel files 은 다음과 같은 내용을 포함한다:

* model weights.
* TensorFlow graph - prediction, training, evaluation에 사용된다.
* model의 구조와 관련된 설정 정보


In [32]:
t = time.time()

savedModel_directory = './{}'.format(int(t))

tf.saved_model.save(model, savedModel_directory)

SavedModel로 저장된 모형은 `tf.saved_model.load(export_dir)` 로 불러들일 수 있다. 

In [33]:
reloaded_SavedModel = tf.saved_model.load(savedModel_directory)

`tf.saved_model.load` 은 Keras object가 아니다. 따라서 `.fit`, `.predict`, `.summary` 등과 같은 methods를 사용할 수 없다. `reloaded_SavedModel` 로 Prediction을 수행하려면 Keras model로 re-load해야 한다.

predictions을 수행하기 위해서 다음과 같이 사용한다.

```python
reloaded_SavedModel(image_batch, training=False)
```


In [35]:
for image_batch, label_batch in testing_batches.take(1):
    prediction_1 = model.predict(image_batch)
    prediction_2 = reloaded_SavedModel(image_batch, training=False).numpy()
    difference = np.abs(prediction_1 - prediction_2)
    print(difference.max())

2.682209e-07


TensorFlow SavedModel로 부터 Keras model을 가져오기 위해서 `tf.keras.models.load_model` function을 사용한다. 

In [36]:
reloaded_keras_model_from_SavedModel = tf.keras.models.load_model(savedModel_directory)

reloaded_keras_model_from_SavedModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

## Saving Models During Training


훈련중에 validation loss는 훈련이 잘 진행되는지 확인할 수 있는 지표 역할을 한다. overfitting이 일어나지 않는 최적 validation loss의 모형을 저장할 수 있다. 

`tf.keras` 에서 다음과 같은 callback을 사용한다:

```python
tf.keras.callbacks.ModelCheckpoint('./best_model.h5', monitor='val_loss', save_best_only=True)
```

위의 callback은 매 epoch마다 Keras HDF5 file로 저장한다. `save_best_only=True` argument을 주면, 가장 최적의 모형만을 저장한다.

In [37]:
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(256, activation = 'relu'),
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Stop training when there is no improvement in the validation loss for 10 consecutive epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Save the Model with the lowest validation loss
save_best = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',
                                               monitor='val_loss',
                                               save_best_only=True)

history = model.fit(training_batches,
                    epochs = 100,
                    validation_data=validation_batches,
                    callbacks=[early_stopping, save_best])

Epoch 1/100
782/782 [==============================] - 2s 3ms/step - loss: 0.8801 - accuracy: 0.6777 - val_loss: 0.6076 - val_accuracy: 0.7789
Epoch 2/100
782/782 [==============================] - 2s 3ms/step - loss: 0.5680 - accuracy: 0.7948 - val_loss: 0.5342 - val_accuracy: 0.8075
Epoch 3/100
782/782 [==============================] - 2s 2ms/step - loss: 0.5181 - accuracy: 0.8150 - val_loss: 0.5108 - val_accuracy: 0.8109
Epoch 4/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4786 - accuracy: 0.8278 - val_loss: 0.4771 - val_accuracy: 0.8253
Epoch 5/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4505 - accuracy: 0.8367 - val_loss: 0.4629 - val_accuracy: 0.8302
Epoch 6/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4292 - accuracy: 0.8437 - val_loss: 0.4287 - val_accuracy: 0.8443
Epoch 7/100
782/782 [==============================] - 2s 2ms/step - loss: 0.4067 - accuracy: 0.8509 - val_loss: 0.4070 - val_accuracy: 0.8495